In [16]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
def prepare_df():

    file_path = '/Users/julioberrocal/Desktop/Classes/Winter 2024/Advanced Programming/Final Project/Muffintown_Final/Query.xlsx'
    df = pd.read_excel(file_path)

    # Converting the SKU into a String object
    df['Current Job'] = df['Current Job'].astype(str)

    # Converting columns to Datetime object
    df['Start of Batch Date/Time'] = pd.to_datetime(df['Start of Batch Date/Time'])
    df['End of Batch Date/Time'] = pd.to_datetime(df['End of Batch Date/Time'])

    # Calculating the batch length in hours
    df['Batch Length (Hours)'] = (df['End of Batch Date/Time'] - df['Start of Batch Date/Time']).dt.total_seconds() / 3600

    # Creating Units per Hour Column
    df['Units Per Hour'] = df['Batch Count'] / df['Batch Length (Hours)']

    # Remove rows with missing values in 'Optimal Cases Per Hour' column
    df.dropna(subset=['Optimal Cases Per Hour'], inplace=True)

    compliant_df = df[(df['Units Per Hour'] >= df['Lower Target']) & (df['Units Per Hour'] <= df['Upper Target'])]
    non_compliant_df = df[(df['Units Per Hour'] < df['Lower Target']) | (df['Units Per Hour'] > df['Upper Target'])]

    total_observations = len(df)
    print(total_observations)
    pct_compliant = 100 * round(len(compliant_df)/total_observations,2)
    pct_non_compliant = 100 * round(len(non_compliant_df)/total_observations,2)

    print(f'The number of compliant observations is: {len(compliant_df)} or {pct_compliant} percent.')
    print(f'The number of non-compliant observations is: {len(non_compliant_df)} or {pct_non_compliant} percent.')

    return df

initial_df = prepare_df()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/julioberrocal/Desktop/Classes/Winter 2024/Advanced Programming/Final Project/Muffintown_Final/Query.xlsx'

In [19]:
def analyze_sku(df, skus):
    analysis_results = []
    
    for sku in skus:

        # Filter by the current SKU
        filtered_df = df[df['Current Job'] == sku]
        
        # Number of records for the SKU
        records = len(filtered_df)
        
        # Percentage of compliant records for the SKU
        compliant_records = len(filtered_df[(filtered_df['Units Per Hour'] >= filtered_df['Lower Target']) & 
                                                (filtered_df['Units Per Hour'] <= filtered_df['Upper Target'])])
        
        compliance_percentage = round((compliant_records / records),2) * 100 if records != 0 else 0
        
        # Average units per hour of the SKU
        avg_units_per_hour = round(filtered_df['Units Per Hour'].mean(),2)
        
        # Standard deviation of units per hour of the SKU
        stdev_units_per_hour = round(filtered_df['Units Per Hour'].std(),2)

        # Plot histogram of units per hour for the SKU
        plt.figure(figsize=(8, 6))
        plt.hist(filtered_df['Units Per Hour'], bins=20, color='skyblue', edgecolor='black')
        plt.title(f'Units Per Hour Distribution for SKU: {sku}')
        plt.xlabel('Units Per Hour')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.savefig(f'{sku}_histogram.png')

        analysis_results.append({
            'SKU': sku,
            'Number of Records': records,
            'Percentage of Compliant Records': compliance_percentage,
            'Average Units Per Hour': avg_units_per_hour,
            'Standard Deviation of Units Per Hour': stdev_units_per_hour
        })
    
    return analysis_results

skus = ['96605', '24970']
sku_results = analyze_sku(initial_df, skus)
for result in sku_results:
    print(result)

NameError: name 'initial_df' is not defined

In [22]:
def analyze_group(df, group_names):
    analysis_results = []
    
    for group_name in group_names:
        # Filter by the current Group Name
        filtered_df = df[df['Group Name'] == group_name]
        
        # Number of records for the Group Name
        records = len(filtered_df)
        
        # Percentage of compliant records for the Group Name
        compliant_records = len(filtered_df[(filtered_df['Units Per Hour'] >= filtered_df['Lower Target']) & 
                                                (filtered_df['Units Per Hour'] <= filtered_df['Upper Target'])])
        compliance_percentage = round((compliant_records / records),2) * 100 if records != 0 else 0
        
        analysis_results.append({
            'Group Name': group_name,
            'Number of Records': records,
            'Percentage of Compliant Records': compliance_percentage,
        })
    
    return analysis_results

group_names = ['LAWRENCE AUTOBAKE', 'LAWRENCE - SW PACK']
group_results = analyze_group(initial_df, group_names)
for result in group_results:
    print(result)

NameError: name 'initial_df' is not defined

In [24]:
def analyze_sku_and_group(df, skus, group_names):
    analysis_results = []
    
    for sku in skus:
        for group_name in group_names:
            # Filter by the current SKU and Group Name
            filtered_df = df[(df['Current Job'] == sku) & (df['Group Name'] == group_name)]
            
            # Number of records for the SKU and Group Name combination
            num_records = len(filtered_df)
            
            # Percentage of compliant records for the SKU and Group Name combination
            num_compliant_records = len(filtered_df[(filtered_df['Units Per Hour'] >= filtered_df['Lower Target']) & 
                                                    (filtered_df['Units Per Hour'] <= filtered_df['Upper Target'])])
            
            compliance_pct = round((num_compliant_records / num_records) * 100,2) if num_records != 0 else 0
            
            # Average units per hour of the SKU and Group Name combination
            avg_units_per_hour = round(filtered_df['Units Per Hour'].mean(),2)
            
            # Standard deviation of units per hour of the SKU and Group Name combination
            stdev_units_per_hour = round(filtered_df['Units Per Hour'].std(),2)
            
            analysis_results.append({
                'SKU': sku,
                'Group Name': group_name,
                'Number of Records': num_records,
                'Percentage of Compliant Records': compliance_pct,
                'Average Units Per Hour': avg_units_per_hour,
                'Standard Deviation of Units Per Hour': stdev_units_per_hour
            })
    
    return analysis_results

skus = ['96605', '24970']
group_names = ['LAWRENCE AUTOBAKE', 'LAWRENCE - SW PACK']
analysis_results = analyze_sku_and_group(initial_df, skus, group_names)

for result in analysis_results:
    print(result)

NameError: name 'initial_df' is not defined

In [23]:
def process_data():
    return "ToDo"

def upload_file(files):
    file_paths = [file.name for file in files]
    return file_paths

tab_search = gr.Interface(
    inputs = [gr.Textbox(label="SKU"), gr.Textbox(label="Factory")],
    outputs = [gr.Textbox(label="Records"), gr.Textbox(label="Compliance"),gr.Textbox(label="Avg Units/Hr"), gr.Textbox(label="Std Dev"), gr.BarPlot(label="Graph")],
    fn = process_data
)

with gr.Blocks() as tab_files:
    with gr.Row():
        with gr.Column():
            file_output = gr.File()
            upload_button = gr.UploadButton("Click to Upload Reference File", file_types=["files"])
            upload_button.upload(upload_file, upload_button, file_output)
        with gr.Column():
            file_output = gr.File()
            upload_button = gr.UploadButton("Click to Upload Template File", file_types=["files"])
            upload_button.upload(upload_file, upload_button, file_output)
            
    
demo = gr.TabbedInterface([tab_files, tab_search], ["Upload Files", "Search"])

#demo.launch(share=True, auth=("BrianA", "Brian@Muffintown1"))
demo.launch(share=True)

/Users/abdurrahman/anaconda3/lib/python3.11/site-packages/gradio/utils.py:857: UserWarning: Expected 0 arguments for function <function process_data at 0x28f3b1440>, received 2.
  warnings.warn(
/Users/abdurrahman/anaconda3/lib/python3.11/site-packages/gradio/utils.py:865: UserWarning: Expected maximum 0 arguments for function <function process_data at 0x28f3b1440>, received 2.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7887
Running on public URL: https://4cf16884b1c3bfd2a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
